In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
train = pd.read_csv('data/train_ver02.csv', index_col='STU_ID')
y = train.F3ONET2CURR
x = train.drop(columns='F3ONET2CURR')

In [2]:
with open('data/my_val.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)
del data
np.random.seed(123)

In [3]:
test = pd.read_csv('data/test_ver02.csv', index_col='STU_ID')
test = test.drop(columns='F3ONET2CURR')
test.head()

,BYSTUWT,BYSEX,BYRACE,BYSTLANG,BYHOMLNG,BYSTLNG2,BYDOB_P,BYFCOMP,BYPARED,BYMOTHED,...,BYS24C,BYS24D,BYS24E,BYS28,BYS71A,BYS71B,BYS71C,BYS71D,BYS71E,BYS71F
STU_ID,,,,,,,,,,,,,,,,,,,,,
310221,575.7933,2.0,6.0,1.0,1.0,4,198412.0,2.0,4.0,4.0,...,4.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0
228218,281.6741,1.0,7.0,1.0,1.0,4,198507.0,6.0,1.0,1.0,...,3.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
179231,275.0641,1.0,7.0,1.0,1.0,4,198607.0,1.0,4.0,4.0,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
215101,149.2901,2.0,3.0,1.0,1.0,4,198507.0,2.0,3.0,1.0,...,4.0,1.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
245121,186.5067,1.0,7.0,1.0,1.0,4,198601.0,5.0,3.0,3.0,...,2.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0


- baseline modeling

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
pipe = Pipeline([('전처리',MinMaxScaler()), 
                 ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=300))])

In [5]:
pipe.fit(x,y)

Pipeline(steps=[('전처리', MinMaxScaler()),
                ('알고리즘',
                 RandomForestClassifier(max_depth=16, n_estimators=300))])

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, pipe.predict(val_x))

0.26717557251908397

In [6]:
pred = pipe.predict(test)

In [7]:
pred.shape

(2479,)

In [8]:
test.shape

(2479, 151)

In [9]:
submission = pd.DataFrame(pred, index=test.index, columns=['PRED']).reset_index()
submission

,STU_ID,PRED
0,310221,43
1,228218,49
2,179231,43
3,215101,41
4,245121,51
...,...,...
2474,439214,11
2475,106206,13
2476,272131,43
2477,350228,43


In [10]:
submission.to_csv('my_pred22.csv', index=False)

- one-vs-all modeling

In [11]:
my_class = y.unique()
my_class

array([29, 27, 39, 53, 13, 43, 51, 11, 21, 17, 47, 15, 25, 31, 35, 33, 41,
       49, 19, 37], dtype=int64)

In [12]:
my_models = []
for s in my_class:
    target = x.loc[y[y==s].index]
    non_target = x.loc[y[y!=s].index]
    # undersampling코드
    #non_target = x.loc[y[y!=s].index].sample(len(target))
    my_sample = pd.concat([target, non_target])
    my_sample = my_sample.sample(len(my_sample))
    my_y = pd.concat([pd.Series(1, index=y.loc[target.index].index),
                      pd.Series(0, y.loc[non_target.index].index)])
    my_y = my_y.loc[my_sample.index]
    pipe = Pipeline([('전처리',MinMaxScaler()), 
                     ('알고리즘', RandomForestClassifier(max_depth=16, n_estimators=300))])
    pipe.fit(my_sample, my_y)
    my_models.append(pipe)

- Undersampling?
- Pipeline? 최적모델 탐색

In [75]:
#20개 모델에 대한 예측 결과 생성
pred = np.zeros((len(val_x), len(my_class)))
for i, m in enumerate(my_models):
    p = m.predict_proba(val_x)[:,1]
    pred[:,i] = p
pred_class = [my_class[i] for i in pred.argmax(axis=1)]

In [76]:
accuracy_score(val_y, pred_class)

0.2655685014061872

In [77]:
with open('data/one-vs-all.pkl', 'wb') as f:
    joblib.dump({'my_models':my_models},f)

In [13]:
pred = np.zeros((len(test),len(my_class)))
for i, m in enumerate(my_models):
    p = m.predict_proba(test)[:,1]
    pred[:,i] = p
pred_class = [my_class[i] for i in pred.argmax(axis=1)]
pred_class

[43,
 51,
 43,
 43,
 51,
 13,
 11,
 17,
 41,
 43,
 43,
 25,
 13,
 43,
 13,
 43,
 43,
 43,
 43,
 43,
 25,
 13,
 43,
 21,
 13,
 43,
 43,
 43,
 43,
 43,
 41,
 43,
 51,
 11,
 43,
 25,
 29,
 43,
 43,
 17,
 43,
 19,
 43,
 11,
 19,
 43,
 43,
 29,
 35,
 43,
 43,
 43,
 43,
 53,
 15,
 47,
 43,
 43,
 43,
 43,
 43,
 29,
 15,
 43,
 11,
 43,
 51,
 43,
 51,
 43,
 43,
 43,
 43,
 43,
 51,
 43,
 47,
 43,
 43,
 15,
 43,
 27,
 11,
 43,
 35,
 43,
 51,
 11,
 43,
 43,
 51,
 15,
 31,
 35,
 29,
 43,
 43,
 31,
 43,
 29,
 53,
 13,
 29,
 29,
 43,
 29,
 13,
 43,
 25,
 13,
 43,
 51,
 43,
 43,
 29,
 51,
 17,
 43,
 43,
 29,
 43,
 43,
 43,
 43,
 13,
 43,
 17,
 35,
 29,
 29,
 13,
 29,
 13,
 41,
 43,
 43,
 43,
 17,
 25,
 43,
 43,
 47,
 43,
 43,
 43,
 49,
 43,
 29,
 43,
 29,
 13,
 43,
 43,
 43,
 29,
 43,
 43,
 43,
 43,
 25,
 43,
 29,
 13,
 43,
 51,
 43,
 43,
 43,
 43,
 25,
 29,
 51,
 15,
 25,
 53,
 47,
 13,
 11,
 43,
 43,
 29,
 25,
 25,
 43,
 43,
 43,
 41,
 29,
 43,
 43,
 29,
 29,
 43,
 25,
 43,
 51,
 43,
 19,
 29,
 13,


In [14]:
submission = pd.DataFrame(pred_class, index=test.index, columns=['PRED']).reset_index()
submission.to_csv('my_pred23.csv', index=False)